In [1]:
import torch
import pickle

from model_training_ck.models import MobileNetV3

master_models_dict = "model_training_ck/models/master_models.pkl"
model_name = "master"
model_dir = "model_training_ck/distilled_lottery_tickets"

/home/diogoalves/thesis/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open(master_models_dict, 'rb') as f:
    models_dict = pickle.load(f)

/home/diogoalves/thesis/.venv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [3]:
def get_top_models_by_val_loss(models_dict, best_model_amount=7):
    sorted_models = sorted(models_dict.items(), key=lambda item: item[1]['min_val_loss'])

    top_models = dict(sorted_models[:best_model_amount])

    for key, model in top_models.items():
        print(f"Cycle: {model['cycle']}")
        print(f"Min Val Loss: {model['min_val_loss']}")
        print(f"Params: {model['params']}")
        print("-" * 20)

    return top_models

top_models = get_top_models_by_val_loss(models_dict)

Cycle: 10
Min Val Loss: 0.07144093990791589
Params: 590669
--------------------
Cycle: 6
Min Val Loss: 0.08029544888995588
Params: 897098
--------------------
Cycle: 4
Min Val Loss: 0.08177518472075462
Params: 1106117
--------------------
Cycle: 5
Min Val Loss: 0.08246598567347974
Params: 996107
--------------------
Cycle: 7
Min Val Loss: 0.08590757776983082
Params: 807994
--------------------
Cycle: 11
Min Val Loss: 0.08807827555574477
Params: 532205
--------------------
Cycle: 2
Min Val Loss: 0.09089144552126527
Params: 1364157
--------------------


In [4]:
model_index = 11
best_model = models_dict[model_index]["model"]

In [5]:
from model_training_ck.trainer import count_nonzero_parameters

model = MobileNetV3(num_labels=7)
model.load_state_dict(best_model)
count_nonzero_parameters(model)

/home/diogoalves/thesis/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/diogoalves/thesis/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


532205

In [6]:
best_model_cycle = models_dict[model_index]["cycle"]
best_model_params = models_dict[model_index]["params"]

model_name = f"{model_dir}/{model_name}_{best_model_cycle}_{best_model_params}.pt"
torch.save(model.state_dict(), model_name)